In [ ]:
# Some MQTT based main files for the esp8266 sensors in DoESLiverpool
# For cheaper deployment

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

In [7]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB6 --baud=115200 
Ready.


In [ ]:
# Code below is to run just the SI7021 humidity temp sensor
# (and then the dallas temperature string)

In [6]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.30.130
pinled       2
boardname    esp8266B

Sent 5 lines (112 bytes) to config.txt.


In [ ]:
# we can move some of these state functions into another file

In [18]:
%sendtofile main.py

import time, network, machine, ubinascii, esp

fconfig = dict(x.split()  for x in open("config.txt"))

pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
plight = machine.ADC(0)

si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
client.connect()
topicip = fconfig["boardname"]+"/ip"
topicmac = fconfig["boardname"]+"/mac"
topicfreemem = fconfig["boardname"]+"/freemem"
macaddress = ubinascii.hexlify(si.config('mac'),':').decode().upper()

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

# dallas string sensor code; G=Orange, D8=Yellow, 3V3=Blue
import onewire, ds18x20
pindallas = machine.Pin(13, machine.Pin.OUT, machine.Pin.PULL_UP)
dallasobj = 5 # tries
dallasscanned = [ ]
topicds = [ ]
def dallasinitconvert():
    global dallasobj, dallasscanned
    if isinstance(dallasobj, int):
        try:
            dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
        except onewire.OneWireError as e:
            dallasobj -= 1
            return []
        dallasscanned = dallasobj.scan()
        print("dallasscanned", dallasscanned)
        if dallasscanned:
            sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
            dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
            print("DS18B20 n=%d"%len(dallasscanned))
            topicds.extend([ fconfig["boardname"]+"/DS18B20/"+str(i)  for i in range(len(dallasscanned)) ])
    if dallasscanned:
        dallasobj.convert_temp()

# SI7021 humidity sensor code
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
bsi7021init = False
def SI7021humiditytemp():
    global bsi7021init
    if not bsi7021init:
        i2c.writeto(0x40, b'\xFE')
        time.sleep_ms(20)
        bsi7021init = True
    i2c.writeto(0x40, b'\xE5')
    time.sleep_ms(20)   # give it time to take a reading or it fails
    bh = i2c.readfrom(0x40, 2)
    bt = i2c.readfrom_mem(0x40, 0xE0, 2)
    rh = (bh[0]<<8)+(bh[1]&0xFC)
    rt = (bt[0]<<8)+(bt[1]&0xFC)
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

topichumid = fconfig["boardname"]+"/SI7021/humid"
topictemp = fconfig["boardname"]+"/SI7021/temp"
topicldr = fconfig["boardname"]+"/LDR"

tdallasconverttime = 0
t0 = time.ticks_ms()
plightv0 = 0
tipnumbernext = 0
while True:
    t = time.ticks_ms()
    if t > tipnumbernext:
        client.publish(topicip, ipnumber, retain=True, qos=1)
        client.publish(topicmac, macaddress, retain=True, qos=1)
        client.publish(topicfreemem, "%d" % esp.freemem(), retain=True, qos=1)
        tipnumbernext = t + 60000
    
    plightv = plight.read()
    if abs(plightv - plightv0) > 45:
        client.publish(topicldr, "%d"%plightv)
        plightv0 = plightv
        
    if dallasobj and dallasscanned and tdallasconverttime and t > tdallasconverttime + 800:
        ds = [ dallasobj.read_temp(d)  for d in dallasscanned ]
        print(ds)
        for i, t in enumerate(ds):
            if t != 85.0:
                client.publish(topicds[i], "%f"%t)
        tdallasconverttime = 0
    
    if t < t0 + 1000:
        time.sleep_ms(10)
        continue
        
    pled.value(0)
    try:
        humid, temp = SI7021humiditytemp()
        print(humid, temp)
        client.publish(topichumid, "%f"%humid)
        client.publish(topictemp, "%f"%temp)
    except OSError as e:
        print(e)
    pled.value(1)
    
    if dallasobj and t > tdallasconverttime + 800:
        dallasinitconvert()
        tdallasconverttime = t
    t0 = t
    

Sent 119 lines (3789 bytes) to main.py.


In [17]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB5 --baud=115200 
Ready.
